In [79]:
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import numpy as np
import csv
import torch.optim as optim
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset

In [80]:
with open('database_ampop_LossF.csv', 'r') as file:
    reader = csv.reader(file)
    header = next(reader)  # Lê a primeira linha como cabeçalho (opcional)
    data_base = [list(map(float, row)) for row in reader]

data_base = torch.tensor(data_base)
X_medidas = data_base[:,-4:].numpy()
Y_larguras = data_base[:,:-4].numpy()

print(len(X_medidas))
# print(Y_larguras[0,:])
print(X_medidas[0,:])
print(Y_larguras[0,:])

2001
[1.1289453e+03 1.3182567e+07 2.3636074e+07 1.9800040e-01]
[2.5278605e-05 3.5901721e-05 4.0834824e-05 6.3069965e-05 4.7000000e+01]


In [81]:
n_epochs = 2000
size_data_test = 0.3
hidden_size1 = 8
hidden_size2 = 4
learning_rate = 0.001
batch_size = 16

scaler_X = MinMaxScaler()
scaler_Y = MinMaxScaler()

X_normalized = scaler_X.fit_transform(X_medidas)
Y_normalized = scaler_Y.fit_transform(Y_larguras)

X_train, X_val, Y_train, Y_val = train_test_split(X_normalized, Y_normalized, test_size=size_data_test)
print(X_normalized[0,:])

X_train = torch.Tensor(X_train)
X_val = torch.Tensor(X_val)
Y_train = torch.Tensor(Y_train)
Y_val = torch.Tensor(Y_val)

train_dataset = TensorDataset(X_train, Y_train)
train_loader = DataLoader(train_dataset, batch_size=batch_size)

val_dataset = TensorDataset(X_val, Y_val)
val_loader = DataLoader(val_dataset, batch_size=batch_size)

[0.4044183  0.45701337 0.23368546 0.35392308]


In [82]:
class AmpOpT_NN(nn.Module):
    def __init__(self):
        super().__init__()
        self.input_layer = nn.Linear(4,hidden_size1)
        self.h_layer1 = nn.Linear(hidden_size1,hidden_size2)
        self.h_layer2 = nn.Linear(hidden_size2,5)
        self.relu = nn.ReLU()
    
    def forward(self, x):
        out = self.relu(self.input_layer(x))
        out = self.relu(self.h_layer1(out))
        out = self.h_layer2(out)
        return out

In [83]:
AmpOpT_pred_model = AmpOpT_NN()
loss = nn.MSELoss()

optimizer = torch.optim.Adam(AmpOpT_pred_model.parameters(), lr = learning_rate, weight_decay=1e-4)

for epoch in range(n_epochs):
    AmpOpT_pred_model.train()
    train_loss = 0.0

    for X_batch, Y_batch in train_loader:
        output = AmpOpT_pred_model(X_train)
        l = loss(output, Y_train)
        optimizer.zero_grad()
        l.backward()
        optimizer.step()
        train_loss += l.item()

    train_loss /= len(train_loader) 

    AmpOpT_pred_model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for X_val_batch, Y_val_batch in val_loader:
            val_pred = AmpOpT_pred_model(X_val_batch)
            val_loss += loss(val_pred, Y_val_batch).item()
    
    val_loss /= len(val_loader)
    
    if (epoch + 1) % 100 == 0:
        print(f"Epoch {epoch+1}/{n_epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")

Epoch 100/2000, Train Loss: 0.0434, Val Loss: 0.0454
Epoch 200/2000, Train Loss: 0.0427, Val Loss: 0.0445
Epoch 300/2000, Train Loss: 0.0427, Val Loss: 0.0445
Epoch 400/2000, Train Loss: 0.0420, Val Loss: 0.0443
Epoch 500/2000, Train Loss: 0.0420, Val Loss: 0.0443
Epoch 600/2000, Train Loss: 0.0420, Val Loss: 0.0443
Epoch 700/2000, Train Loss: 0.0420, Val Loss: 0.0443
Epoch 800/2000, Train Loss: 0.0420, Val Loss: 0.0443
Epoch 900/2000, Train Loss: 0.0420, Val Loss: 0.0443
Epoch 1000/2000, Train Loss: 0.0420, Val Loss: 0.0443
Epoch 1100/2000, Train Loss: 0.0420, Val Loss: 0.0443
Epoch 1200/2000, Train Loss: 0.0420, Val Loss: 0.0443
Epoch 1300/2000, Train Loss: 0.0420, Val Loss: 0.0443
Epoch 1400/2000, Train Loss: 0.0420, Val Loss: 0.0443
Epoch 1500/2000, Train Loss: 0.0420, Val Loss: 0.0443
Epoch 1600/2000, Train Loss: 0.0420, Val Loss: 0.0443
Epoch 1700/2000, Train Loss: 0.0420, Val Loss: 0.0443
Epoch 1800/2000, Train Loss: 0.0420, Val Loss: 0.0443
Epoch 1900/2000, Train Loss: 0.0420, 

In [84]:
def predict(model, measures):
    model.eval()
    measures_normalized = scaler_X.transform([measures])
    measures_tensor = torch.tensor(measures_normalized, dtype=torch.float32)
    predictions = model(measures_tensor).detach().numpy()
    return scaler_Y.inverse_transform(predictions)

In [85]:
print('Ganho DC: Max =', max(X_medidas[:,0]), '; Min =', min(X_medidas[:,0]))
print('Freq Corte: Max =', max(X_medidas[:,1]), '; Min =', min(X_medidas[:,1]))
print('Slew Rate: Max =', max(X_medidas[:,2]), '; Min =', min(X_medidas[:,2]))
print('Custo: Max =', max(X_medidas[:,3]), '; Min =', min(X_medidas[:,3]))

Ganho DC: Max = 2789.3887 ; Min = 1.4531754
Freq Corte: Max = 27542288.0 ; Min = 1096478.2
Slew Rate: Max = 97769620.0 ; Min = 1029260.0
Custo: Max = 0.43993607 ; Min = 0.06546725


In [90]:
novas_medidas = [1.5e3, 8e6, 10e6, 0.1]
larguras_previstas = predict(AmpOpT_pred_model, novas_medidas)
print("Larguras de canal previstas:", larguras_previstas)

Larguras de canal previstas: [[2.8612694e-05 1.5153638e-05 3.4545010e-05 2.2851353e-05 2.2770475e+01]]
